# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [78]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
print(sinasc.drop_duplicates().shape)
print('Quantidade de linhas duplicadas:',sinasc.shape[0] - sinasc.drop_duplicates().shape[0])

(27028, 69)
(27028, 69)
Quantidade de linhas duplicadas: 0


In [79]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [80]:
# 3) seu código aqui
sinasc_interesse = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_interesse.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [81]:
# 4) seu código aqui
sinasc_interesse_filtro = sinasc_interesse.drop(sinasc_interesse[sinasc_interesse['APGAR5'].isna()].index)
sinasc_interesse_filtro[sinasc_interesse_filtro['ESCMAE'].isna()]

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
38,1,25,1.0,NaN,3.0,37 a 41 semanas,Única,4,8.0
62,1,20,NaN,NaN,1.0,37 a 41 semanas,Única,4,9.0
66,1,23,5.0,NaN,1.0,37 a 41 semanas,Única,4,10.0
93,1,22,5.0,NaN,2.0,37 a 41 semanas,Única,3,9.0
229,1,25,1.0,NaN,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
26605,1,18,5.0,NaN,1.0,37 a 41 semanas,Única,4,10.0
26765,1,30,2.0,NaN,2.0,37 a 41 semanas,Única,4,8.0
26902,1,22,5.0,NaN,1.0,37 a 41 semanas,Única,3,10.0
26928,1,36,1.0,NaN,3.0,32 a 36 semanas,Dupla,3,9.0


In [82]:
# 5) seu código aqui
sinasc_interesse_filtro['ESTCIVMAE'].fillna(9, inplace=True)
sinasc_interesse_filtro['CONSULTAS'].fillna(9, inplace=True)

In [83]:
# 6) Seu código aqui
sinasc_interesse_filtro['QTDFILVIVO'].fillna(0, inplace=True)

In [84]:
# 7.1.1) seu código aqui
#Verificando a frequência que cada valor aparece
coluna = 'ESCMAE'
lista_coluna = list(sinasc_interesse_filtro[coluna].unique())
qtde_coluna = list(map(lambda x: sinasc_interesse_filtro[sinasc_interesse_filtro[coluna]==x].shape[0], lista_coluna))
dict_coluna = dict(zip(lista_coluna, qtde_coluna))

lista_coluna_freq = list(map(lambda x: x / 26925, dict_coluna.values()))
dict_coluna_freq = dict(zip(dict_coluna.keys(), lista_coluna_freq))
dict_coluna_freq

{'8 a 11 anos': 0.5786815227483751,
 '12 anos ou mais': 0.21381615598885795,
 '1 a 3 anos': 0.01894150417827298,
 '4 a 7 anos': 0.17556174558960075,
 nan: 0.0,
 'Nenhuma': 0.0014856081708449396}

In [85]:
# 7.1.2) seu código aqui
#Verificando o impacto na frequência com preenchimento ffill (forward)
coluna = 'ESCMAE'

sinasc_interesse_filtro_ffill = sinasc_interesse_filtro.copy(deep=True)
sinasc_interesse_filtro_ffill[coluna].fillna(method='ffill', inplace=True)

lista_coluna_ffill = list(sinasc_interesse_filtro_ffill[coluna].unique())
qtde_coluna_ffill = list(map(lambda x: sinasc_interesse_filtro_ffill[sinasc_interesse_filtro_ffill[coluna]==x].shape[0], lista_coluna_ffill))
dict_coluna_ffill = dict(zip(lista_coluna_ffill, qtde_coluna_ffill))

lista_coluna_freq_ffill = list(map(lambda x: x / 26925, dict_coluna_ffill.values()))
dict_coluna_ffill_freq = dict(zip(dict_coluna_ffill.keys(), lista_coluna_freq_ffill))
dict_coluna_ffill_freq

{'8 a 11 anos': 0.5840297121634169,
 '12 anos ou mais': 0.21756731662024142,
 '1 a 3 anos': 0.019090064995357474,
 '4 a 7 anos': 0.17782729805013928,
 'Nenhuma': 0.0014856081708449396}

Adotado para a variável ESCMAE o preenchimento ffill (forward) por manter a frequência relativa muito próxima.

In [86]:
# 7.2.1) seu código aqui
#Verificando a frequência que cada valor aparece
coluna = 'GESTACAO'
lista_coluna = list(sinasc_interesse_filtro[coluna].unique())
qtde_coluna = list(map(lambda x: sinasc_interesse_filtro[sinasc_interesse_filtro[coluna]==x].shape[0], lista_coluna))
dict_coluna = dict(zip(lista_coluna, qtde_coluna))

lista_coluna_freq = list(map(lambda x: x / 26925, dict_coluna.values()))
dict_coluna_freq = dict(zip(dict_coluna.keys(), lista_coluna_freq))
dict_coluna_freq

{'37 a 41 semanas': 0.8365088207985144,
 '42 semanas e mais': 0.025923862581244198,
 '32 a 36 semanas': 0.08297121634168989,
 '22 a 27 semanas': 0.003156917363045497,
 '28 a 31 semanas': 0.0060167130919220055,
 nan: 0.0,
 'Menos de 22 semanas': 0.00025998142989786443}

In [87]:
# 7.2.2) seu código aqui
#Verificando o impacto na frequência com preenchimento ffill (forward)
coluna = 'GESTACAO'

sinasc_interesse_filtro_ffill = sinasc_interesse_filtro.copy(deep=True)
sinasc_interesse_filtro_ffill[coluna].fillna(method='ffill', inplace=True)

lista_coluna_ffill = list(sinasc_interesse_filtro_ffill[coluna].unique())
qtde_coluna_ffill = list(map(lambda x: sinasc_interesse_filtro_ffill[sinasc_interesse_filtro_ffill[coluna]==x].shape[0], lista_coluna_ffill))
dict_coluna_ffill = dict(zip(lista_coluna_ffill, qtde_coluna_ffill))

lista_coluna_freq_ffill = list(map(lambda x: x / 26925, dict_coluna_ffill.values()))
dict_coluna_ffill_freq = dict(zip(dict_coluna_ffill.keys(), lista_coluna_freq_ffill))
dict_coluna_ffill_freq

{'37 a 41 semanas': 0.8753946146703807,
 '42 semanas e mais': 0.02688950789229341,
 '32 a 36 semanas': 0.08761374187558031,
 '22 a 27 semanas': 0.0033054781801299905,
 '28 a 31 semanas': 0.006462395543175487,
 'Menos de 22 semanas': 0.0003342618384401114}

Adotado para a variável GESTACAO o preenchimento ffill (forward) por distribuir a frequência relativa dentro de uma proporcionalidade aceitável.

In [88]:
# 7.3.1) seu código aqui
#Verificando a frequência que cada valor aparece
coluna = 'GRAVIDEZ'
lista_coluna = list(sinasc_interesse_filtro[coluna].unique())
qtde_coluna = list(map(lambda x: sinasc_interesse_filtro[sinasc_interesse_filtro[coluna]==x].shape[0], lista_coluna))
dict_coluna = dict(zip(lista_coluna, qtde_coluna))

lista_coluna_freq = list(map(lambda x: x / 26925, dict_coluna.values()))
dict_coluna_freq = dict(zip(dict_coluna.keys(), lista_coluna_freq))
dict_coluna_freq

{'Única': 0.9788672237697308,
 'Dupla': 0.018124419684308262,
 nan: 0.0,
 'Tríplice e mais': 0.00018570102135561745}

In [89]:
# 7.3.2) seu código aqui
#Verificando o impacto na frequência com preenchimento ffill (forward)
coluna = 'GRAVIDEZ'

sinasc_interesse_filtro_ffill = sinasc_interesse_filtro.copy(deep=True)
sinasc_interesse_filtro_ffill[coluna].fillna(method='ffill', inplace=True)

lista_coluna_ffill = list(sinasc_interesse_filtro_ffill[coluna].unique())
qtde_coluna_ffill = list(map(lambda x: sinasc_interesse_filtro_ffill[sinasc_interesse_filtro_ffill[coluna]==x].shape[0], lista_coluna_ffill))
dict_coluna_ffill = dict(zip(lista_coluna_ffill, qtde_coluna_ffill))

lista_coluna_freq_ffill = list(map(lambda x: x / 26925, dict_coluna_ffill.values()))
dict_coluna_ffill_freq = dict(zip(dict_coluna_ffill.keys(), lista_coluna_freq_ffill))
dict_coluna_ffill_freq

{'Única': 0.981652739090065,
 'Dupla': 0.018161559888579388,
 'Tríplice e mais': 0.00018570102135561745}

Também adotado para a variável GRAVIDEZ o preenchimento ffill (forward) por manter a frequência relativa sem variações significativas.

In [110]:
# 8) seu código aqui
sinasc_interesse_filtro.loc[sinasc_interesse_filtro['APGAR5'] >= 8, 'apgar5_categ'] = 'normal'
sinasc_interesse_filtro.loc[(sinasc_interesse_filtro['APGAR5'] >= 6) & (sinasc_interesse_filtro['APGAR5'] <= 7), 'apgar5_categ'] = 'asfixia leve'
sinasc_interesse_filtro.loc[(sinasc_interesse_filtro['APGAR5'] >= 4) & (sinasc_interesse_filtro['APGAR5'] <= 5), 'apgar5_categ'] = 'asfixia moderada'
sinasc_interesse_filtro.loc[sinasc_interesse_filtro['APGAR5'] <= 3, 'apgar5_categ'] = 'asfixia severa'

apgar5_categ_dummies = pd.get_dummies(sinasc_interesse_filtro['apgar5_categ'])
print('A frequência relativa (%) é:\n', apgar5_categ_dummies.sum()/apgar5_categ_dummies.shape[0]*100)

A frequência relativa (%) é:
 asfixia leve         1.188487
asfixia moderada     0.252553
asfixia severa       0.274838
normal              98.284123
dtype: float64


In [114]:
# 9) seu código aqui
sinasc_interesse_filtro.rename(columns={'LOCNASC':'local_nasc', 'IDADEMAE':'idade_mae', 'ESTCIVMAE':'estado_civil_mae', 'ESCMAE':'escolaridade_mae', 'QTDFILVIVO':'qtde_filho_vivo', 'GESTACAO':'gestacao', 'GRAVIDEZ':'gravidez', 'CONSULTAS':'consultas', 'APGAR5':'apgar5'}, inplace=True)
sinasc_interesse_filtro

,local_nasc,idade_mae,estado_civil_mae,escolaridade_mae,qtde_filho_vivo,gestacao,gravidez,consultas,apgar5,apgar5_categ
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [ ]:
# 9) seu código aqui
